In [1]:
import pandas as pd
import geopandas as gpd
from siuba import *
import shared_utils

from calitp_data_analysis import get_fs
from segment_speed_utils import helpers, time_series_utils, gtfs_schedule_wrangling
from segment_speed_utils.project_vars import SCHED_GCS, SEGMENT_GCS, GTFS_DATA_DICT, analysis_date

i'd start with 5, checking if the columns you want are already there in this file (saved out at the end of stop_arrivals_to_speed, and is a speeds by trip table for that day)

In [2]:
catalog = shared_utils.catalog_utils.get_catalog('gtfs_analytics_data')

In [3]:
catalog.speedmap_segments

{'dir': '${gcs_paths.SEGMENT_GCS}', 'stage1': '${speeds_tables.vp_dwell}', 'proxy_stop_times': 'stop_time_expansion/speedmap_stop_times', 'stage2': 'nearest/nearest_vp_speedmap_proxy', 'stage2b': 'nearest/nearest2_vp_speedmap_proxy', 'stage3': 'speedmap/stop_arrivals_proxy', 'stage3b': 'speedmap/stop_arrivals', 'stage4': 'speedmap/speeds', 'trip_stop_cols': ['trip_instance_key', 'stop_sequence', 'stop_sequence1'], 'shape_stop_cols': ['shape_array_key', 'shape_id'], 'stop_pair_cols': ['stop_pair', 'stop_pair_name', 'segment_id'], 'route_dir_cols': ['route_id', 'route_short_name'], 'segments_file': 'segment_options/speedmap_segments', 'shape_stop_single_segment': 'rollup_singleday/speeds_shape_speedmap_segments', 'shape_stop_single_segment_detail': 'rollup_singleday/speeds_shape_speedmap_segments_detail', 'route_dir_single_segment': 'rollup_singleday/speeds_route_dir_speedmap_segments', 'route_dir_multi_segment': 'rollup_multiday/speeds_route_dir_speedmap_segments', 'min_trip_minutes': '

In [4]:
analysis_date

'2024-10-16'

In [5]:
# path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.stage4}_{analysis_date}.parquet'

In [6]:
path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.shape_stop_single_segment_detail}_{analysis_date}.parquet'

In [7]:
path

'gs://calitp-analytics-data/data-analyses/rt_segment_speeds/rollup_singleday/speeds_shape_speedmap_segments_detail_2024-10-16.parquet'

In [8]:
detail = gpd.read_parquet(path)

In [9]:
detail >> head(3)

,schedule_gtfs_dataset_key,shape_array_key,shape_id,route_id,route_short_name,stop_pair,stop_pair_name,segment_id,time_of_day,p50_mph,n_trips,p20_mph,p80_mph,name,caltrans_district,organization_source_record_id,organization_name,base64_url,geometry
0,0139b1253130b33adcd4b3a4490530d2,51530f7b5ba4cdde7c644c14f262005c,b0c096a2-4c32-4675-b005-43b58b3781e6,874b0dfa-ab3d-47da-ac75-5fd7b9bdea42,C30 WB,0d1ef8b5-90e4-40de-b37b-504cb862d18b__5b9a7352...,Woodlake Park EB__Cypress St & Naranjo Blvd WB,0d1ef8b5-90e4-40de-b37b-504cb862d18b-5b9a7352-...,AM Peak,17.27,4,15.20,20.96,TCRTA TripShot Schedule,06 - Fresno,rec4YWVqVwJ9XqDbq,Tulare County Regional Transit Agency,aHR0cHM6Ly90Y3J0YS50cmlwc2hvdC5jb20vdjEvZ3Rmcy...,"LINESTRING (-119.09762 36.41481, -119.09762 36..."
1,0139b1253130b33adcd4b3a4490530d2,51530f7b5ba4cdde7c644c14f262005c,b0c096a2-4c32-4675-b005-43b58b3781e6,874b0dfa-ab3d-47da-ac75-5fd7b9bdea42,C30 WB,0d1ef8b5-90e4-40de-b37b-504cb862d18b__5b9a7352...,Woodlake Park EB__Cypress St & Naranjo Blvd WB,0d1ef8b5-90e4-40de-b37b-504cb862d18b-5b9a7352-...,Midday,20.38,6,18.46,23.02,TCRTA TripShot Schedule,06 - Fresno,rec4YWVqVwJ9XqDbq,Tulare County Regional Transit Agency,aHR0cHM6Ly90Y3J0YS50cmlwc2hvdC5jb20vdjEvZ3Rmcy...,"LINESTRING (-119.09762 36.41481, -119.09762 36..."
2,0139b1253130b33adcd4b3a4490530d2,51530f7b5ba4cdde7c644c14f262005c,b0c096a2-4c32-4675-b005-43b58b3781e6,874b0dfa-ab3d-47da-ac75-5fd7b9bdea42,C30 WB,0d1ef8b5-90e4-40de-b37b-504cb862d18b__5b9a7352...,Woodlake Park EB__Cypress St & Naranjo Blvd WB,0d1ef8b5-90e4-40de-b37b-504cb862d18b-5b9a7352-...,PM Peak,23.90,5,22.49,25.11,TCRTA TripShot Schedule,06 - Fresno,rec4YWVqVwJ9XqDbq,Tulare County Regional Transit Agency,aHR0cHM6Ly90Y3J0YS50cmlwc2hvdC5jb20vdjEvZ3Rmcy...,"LINESTRING (-119.09762 36.41481, -119.09762 36..."


In [25]:
path = f'{catalog.schedule_downloads.dir}{catalog.schedule_downloads.trips}_{analysis_date}.parquet'

In [26]:
path

'gs://calitp-analytics-data/data-analyses/rt_delay/compiled_cached_views/trips_2024-10-16.parquet'

In [27]:
trips = pd.read_parquet(path)

In [28]:
df = gtfs_schedule_wrangling.get_sched_trips_hr(analysis_date)

In [29]:
df >> head(3)

,route_id,direction_id,shape_id,time_of_day,schedule_gtfs_dataset_key,n_trips,trips_hr
0,001,0.0,0010111,AM Peak,9809d3f8121513057bc5cb8de7b54ce2,3,1.000000
1,001,0.0,0010111,Early AM,9809d3f8121513057bc5cb8de7b54ce2,1,0.333333
2,001,0.0,0010111,Midday,9809d3f8121513057bc5cb8de7b54ce2,5,1.000000


## Where'd the J line go??

In [30]:
df >> filter(_.route_id.str.contains('910'))

,route_id,direction_id,shape_id,time_of_day,schedule_gtfs_dataset_key,n_trips,trips_hr
15680,910-13183,0.0,9100214_JUNE24,AM Peak,0666caf3ec1ecc96b74f4477ee4bc939,19,6.333333
15681,910-13183,0.0,9100214_JUNE24,Early AM,0666caf3ec1ecc96b74f4477ee4bc939,15,5.000000
15682,910-13183,0.0,9100214_JUNE24,Evening,0666caf3ec1ecc96b74f4477ee4bc939,9,1.800000
15683,910-13183,0.0,9100214_JUNE24,Midday,0666caf3ec1ecc96b74f4477ee4bc939,22,4.400000
15684,910-13183,0.0,9100214_JUNE24,Owl,0666caf3ec1ecc96b74f4477ee4bc939,4,1.000000
15685,910-13183,0.0,9100214_JUNE24,PM Peak,0666caf3ec1ecc96b74f4477ee4bc939,32,6.400000
15686,910-13183,0.0,9100215_JUNE24,AM Peak,0666caf3ec1ecc96b74f4477ee4bc939,7,2.333333
15687,910-13183,0.0,9100215_JUNE24,Early AM,0666caf3ec1ecc96b74f4477ee4bc939,8,2.666667
15688,910-13183,0.0,9100215_JUNE24,Evening,0666caf3ec1ecc96b74f4477ee4bc939,1,0.200000
15689,910-13183,0.0,9100215_JUNE24,Midday,0666caf3ec1ecc96b74f4477ee4bc939,11,2.200000


In [40]:
catalog.speedmap_segments

{'dir': '${gcs_paths.SEGMENT_GCS}', 'stage1': '${speeds_tables.vp_dwell}', 'proxy_stop_times': 'stop_time_expansion/speedmap_stop_times', 'stage2': 'nearest/nearest_vp_speedmap_proxy', 'stage2b': 'nearest/nearest2_vp_speedmap_proxy', 'stage3': 'speedmap/stop_arrivals_proxy', 'stage3b': 'speedmap/stop_arrivals', 'stage4': 'speedmap/speeds', 'trip_stop_cols': ['trip_instance_key', 'stop_sequence', 'stop_sequence1'], 'shape_stop_cols': ['shape_array_key', 'shape_id'], 'stop_pair_cols': ['stop_pair', 'stop_pair_name', 'segment_id'], 'route_dir_cols': ['route_id', 'route_short_name'], 'segments_file': 'segment_options/speedmap_segments', 'shape_stop_single_segment': 'rollup_singleday/speeds_shape_speedmap_segments', 'shape_stop_single_segment_detail': 'rollup_singleday/speeds_shape_speedmap_segments_detail', 'route_dir_single_segment': 'rollup_singleday/speeds_route_dir_speedmap_segments', 'route_dir_multi_segment': 'rollup_multiday/speeds_route_dir_speedmap_segments', 'min_trip_minutes': '

In [37]:
path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.shape_stop_single_segment_detail}_{analysis_date}.parquet'

In [41]:
path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.route_dir_single_segment}_{analysis_date}.parquet'

In [42]:
speedmap_segs = gpd.read_parquet(path) #  aggregated

In [43]:
speedmap_segs >> filter(_.route_id.str.contains('910'))

,schedule_gtfs_dataset_key,route_id,route_short_name,stop_pair,stop_pair_name,segment_id,time_period,p50_mph,n_trips,p20_mph,p80_mph,name,caltrans_district,organization_source_record_id,organization_name,base64_url,geometry


it has no route_short_name...
```
857-13183,857,Metro Local Line,C & K LINE LINK,3,,,
901-13183,,Metro G Line 901,METRO G LINE,3,FC4C02,FFFFFF,https://www.metro.net/g-line/
910-13183,,Metro J Line 910/950,METRO J LINE,3,ADB8BF,000000,https://www.metro.net/j-line/
```